In [1]:
import numpy as np

# 第２ステージ　自然なコードで表現する　

## ステップ１１　可変長の引数（順伝播編）

### 11.1 Functionクラスの修正
現状のFunctionクラスを、複数の入出力に対応できるよう修正する。<br>
以下、現状のFunctionクラス
```Python
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()
```

In [2]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [3]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [4]:
class Function:
    def __call__(self, inputs):
        xs = [input.data for  input in inputs]  # Get data from Variable
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]  # Wrap data

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


### 11.2 Addクラスの実装

In [5]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 出力をタプルにする

In [6]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs) # ysはタプル
y = ys[0]
print(y.data)

5


## ステップ１２　可変長の引数（改善編）

### 12.1 １つ目の改善：関数を使いやすく
まずは、関数を<strong>使う人</strong>にとっての改善を行う。<br>
- 現状のコード<br>
```Python
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
```
- 改善後のコード<br>
```Python
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)
```

In [7]:
class Function:
    def __call__(self, *inputs): # ①アスタリスクを付ける
        xs = [input.data for input in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        # ②リストの要素が１つの時は最初の要素を返す
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

- memo<br>
<strong>可変長引数</strong><br>
変数にアスタリスクを付けることで、リストを使わずに任意の数の引数を与えて、<br>
関数を呼ぶことができる。以下に具体例を挙げる。

In [8]:
def print_x(*x):
    print(x)
    
print_x(1, 2, 3)

a, b = (1, 2), (3, 4)
print_x(a, b)

(1, 2, 3)
((1, 2), (3, 4))


In [9]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 出力をタプルにする

In [10]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


### 12.2 ２つ目の改善：関数を実装しやすく
今度は、関数を<strong>実装する人</strong>にとっての改善を行う。<br>
具体的には、メソッドの引数は直接受け取り、結果の変数も直接返す。<br>
- 現状のコード
```Python
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 出力をタプルにする
```
- 改善後のコード
```Python
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
```

In [11]:
class Function:
    def __call__(self, *inputs):
        xs = [input.data for input in inputs]
        ys = self.forward(*xs) # ①アスタリスクを付けてアンパッキング
        if not isinstance(ys, tuple): # ②タプルではない場合の追加対応
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 12.3 add関数の実装
Addクラスを<strong>Pythonの関数</strong>として利用できるようにする。

In [12]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

In [13]:
def add(x0, x1):
    return Add()(x0, x1)

In [14]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


## ステップ１３　可変長の引数（逆伝播編）

### 13.1 可変長引数に対応したAddクラスの逆伝播
ステップ１２の通り、足し算の順伝播は入力が２つ、出力が１つである。<br>
逆伝播時は、逆に、入力が１つ、出力が２つになる。<br>
例えば、$y=x_0+x_1$の場合、
$$\frac{\partial y}{\partial x_0} = 1, \frac{\partial y}{\partial x_1} = 1$$
であることから、足し算の逆伝播は、上流の微分を<strong>そのまま流す</strong>といいことがわかる。

In [15]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

### 13.2 Variableクラスの修正
- 現状のVariableクラス
```Py
class Variable:
    ...
    def backward(self):
    if self.grad is None:
        self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
        f = funcs.pop()
        x, y = f.input, f.output # ①関数の入出力を取得 
        x.grad = f.backward(y.grad) # ②backwardメソッドを呼ぶ

        if x.creator is not None:
            funcs.append(x.creator)
```
現状では、①において、関数の入出力を１つだけに限定している。<br>
これを複数の変数に対応できるように4つの修正を加える。

In [16]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] # ①
            gxs = f.backward(*gys) # ②
            if not isinstance(gxs, tuple): # ③
                gxs = (gxs,)

            for input, gx in zip(f.inputs, gxs): # ④
                input.grad = gx

                if input.creator is not None:
                    funcs.append(input.creator)

①の箇所で、出力変数であるoutputsの微分をリストにまとめる。<br>
そして②において、関数fの逆伝播を呼び出す。ここで、変数に*をつけて、<br>
リストのアンパック（展開）を行う。<br>
また、③の箇所では、gxsがタプルではない時に、それをタプルへと変換する。
<br>
④では、各入力とそれに対応する微分のペアを設定する。


### 13.3 Squareクラスの実装

In [17]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 修正前はx = self.input.data
        gx = 2 * x * gy
        return gx

In [18]:
def square(x):
    f = Square()
    return f(x)

In [19]:
def add(x0, x1):
    return Add()(x0, x1)

In [20]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


- memo<br>
上記のプログラムでは、$z=x^2+y^2$という計算を行った。<br>
`z.backward()`の部分の計算過程を図にしたものが以下のものである。<br>
![fit13_3](./image/step13_ex.png)

## ステップ１４ 同じ変数を繰り返し使う
現状では、同じ変数を使って足し算を行うと、微分が正しく計算できない。<br>
そのため改善する。
以下の例では、$y=x+x$の時、$\frac{\partial y}{\partial x}=2$であるが、<br>
プログラムでは、1が出力されている。

In [21]:
x = Variable(np.array(3.0))
y = add(x, x)
print(f'y: {y.data}')

y.backward()
print(f'x.grad: {x.grad}')

y: 6.0
x.grad: 1.0


### 14.1 問題の原因
```Python
class Variable:
    ...
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx # ここが間違い！

                if x.creator is not None:
                    funcs.append(x.creator)
```
現状の実装では、同じ変数を繰り返し使った場合、伝播する微分の値が置き換わってしまう。<br>
正しくは、伝播する微分の<strong>和</strong>を求める必要がある。

### 14.2 解決策

In [22]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for input, gx in zip(f.inputs, gxs):
                if input.grad is None:
                    input.grad = gx # 初回の微分時にはそのまま代入
                else:
                    input.grad = input.grad + gx # 2回目以降では、微分を加算する

                if input.creator is not None:
                    funcs.append(input.creator)

In [23]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

2.0


In [24]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad)

3.0


### 14.3 微分をリセットする
これで解決かと思いきや、まだ注意すべき点がある。<br>
同じ変数を使って、別の計算を行う場合に、期待する動作をしない場合がある。

In [25]:
# １回目の計算
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# ２回目の計算（同じxを使って、別の計算を行う。）
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
5.0


In [26]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self): # 微分をリセットするメソッドの追加
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for input, gx in zip(f.inputs, gxs):
                if input.grad is None:
                    input.grad = gx
                else:
                    input.grad = input.grad + gx

                if input.creator is not None:
                    funcs.append(input.creator)

In [27]:
# １回目の計算
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# ２回目の計算（同じxを使って、別の計算を行う。）
x.cleargrad() # 微分のリセット
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


## ステップ１５ 複雑な計算グラフ（理論編）
省略

## ステップ16 複雑な計算グラフ（実装編）
### 16.1 世代の追加
インスタンス変数のgenerationで、どの<strong>世代</strong>の関数であるかを示す。

In [28]:
class Function:
    def __call__(self, *inputs):
        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([input.generation for input in inputs]) # 世代を決定する
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 16.2 世代順に取り出す
世代順に関数を取り出すためには、generationの値が大きい順に、関数を取り出せば良い。<br>
以下、ダミーの関数を使って、簡単な実験を行う。

In [29]:
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
    f = Function() # ダミーの関数クラス
    f.generation = g
    funcs.append(f)
    
[f.generation for f in funcs]

[2, 0, 1, 4, 2]

毎回ソートするのは、計算の無駄があるのでは？<br>
世代の最大値をソートせずに抽出した方が速いのでは？<br>
と思ったので、簡単に比較してみる。

In [30]:
# 原著の実装法
funcs.sort(key=lambda x: x.generation)
print([f.generation for f in funcs])

f = funcs.pop()
f.generation

[0, 1, 2, 2, 4]


4

In [31]:
# sortせずに出してみる
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
    f = Function() # ダミーの関数クラス
    f.generation = g
    funcs.append(f)
    
f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
f.generation

4

どっちが速いか確かめてみる。<br>
リストの数によらず、sortしない方が速そう<br>
[このリンク](https://qiita.com/Hironsan/items/68161ee16b1c9d7b25fb)によると、データ数を$n$とした時、sort関数の計算量は$O(n \ log \ n)$、<br>
max関数の計算量は$O(n)$であるため、sortは使わない方が良さそう

In [32]:
import time
test_generations = np.random.randint(0, 100, 3).tolist()
funcs = []

for g in test_generations:
    f = Function() # ダミーの関数クラス
    f.generation = g
    funcs.append(f)

start = time.time()
funcs.sort(key=lambda x: x.generation)
#print([f.generation for f in funcs])

f = funcs.pop()
end = time.time() - start
print(f'f.generation: {f.generation}, time: {end: .6f}')

f.generation: 73, time:  0.000105


In [33]:
funcs = []

for g in test_generations:
    f = Function() # ダミーの関数クラス
    f.generation = g
    funcs.append(f)

start = time.time() 
f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
end = time.time() - start
print(f'f.generation: {f.generation}, time: {end: .6f}')

f.generation: 73, time:  0.000266


### 16.3 Variableクラスのbackward

In [34]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 # 変数の宣言

    def set_creator(self, func):
        self.creator = func
        # 生みの親の関数を記憶するときに、親の関数より１大きい
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)

        add_func(self.creator)

        while funcs:
            # 世代の最大値をもつ関数を取得
            f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for input, gx in zip(f.inputs, gxs):
                if input.grad is None:
                    input.grad = gx
                else:
                    input.grad = input.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

### 16.4 動作確認 

In [35]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

$y=(x^2)^2 + (x^2)^2$の時、$y'=(2x^4)'=8x^3$より、$x=2.0$の時、微分は$64.0$となる。

In [36]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
None


## ステップ１７ メモリ管理と循環参照

### 17.2 参照カウント方式のメモリ管理

In [37]:
class obj:
    pass

def f(x):
    print(x)

In [38]:
a = obj() # 参照カウント１
f(a) # 関数の中では参照カウント２
# 関数を抜けると参照カウント１
a = None # 参照カウント０

In [39]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c
a = b = c = None

### 17.3 循環参照

In [40]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c
c.a = a

a = b = c = None

### 17.4 weakrefモジュール

jupyter形式では、ユーザの裏側で追加の参照を保持するため、通常のpythonファイルで実行してみる

In [41]:
!python ./src/step17_4.py

before: <weakref at 0x7f0da554f8f0; to 'numpy.ndarray' at 0x7f0da67a0f90>
[1 2 3]
after: <weakref at 0x7f0da554f8f0; dead>


### 17.5 動作確認
weakref適用後では、メモリ使用量が増加していないことがわかる。<br>

In [42]:
!python ./src/step17_5.py

weakref適用前
Filename: ./src/step17_5.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    98  54.1797 MiB  54.1797 MiB           1   @profile(precision=4)
    99                                         def test_before(size):
   100  54.8750 MiB   0.0000 MiB          11       for i in range(10):
   101  54.8750 MiB   0.2891 MiB          10           x = Variable_(np.random.randn(size))  # big data
   102  54.8750 MiB   0.4062 MiB          10           y = square(square(square(x)))


weakref適用後
Filename: ./src/step17_5.py

Line #    Mem usage    Increment  Occurrences   Line Contents
   190  54.8750 MiB  54.8750 MiB           1   @profile(precision=4)
   191                                         def test_after(size):
   192  54.8750 MiB   0.0000 MiB          11       for i in range(10):
   193  54.8750 MiB   0.0000 MiB          10           x = Variable(np.random.randn(size))  # big data
   194  54.8750 MiB   0.0000 MiB          10           y = square(square(square(x))

## ステップ18 メモリ消費量を減らすモード

### 18.1 不要な微分は保持しない

In [43]:
import weakref

In [44]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)

        add_func(self.creator)

        while funcs:
            f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for input, gx in zip(f.inputs, gxs):
                if input.grad is None:
                    input.grad = gx
                else:
                    input.grad = input.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

In [45]:
class Function:
    def __call__(self, *inputs):
        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([input.generation for input in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [46]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

In [47]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()
print(y.grad, t.grad)  # None None
print(x0.grad, x1.grad)  # 2.0 1.0

1.0 1.0
1.0 None


多くの場合、`y`や`t`などの中間の変数については、その微分を必要とするケースは多くない。<br>
そこで、微分を消去するモードを追加する。

In [48]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)

        add_func(self.creator)

        while funcs:
            f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for input, gx in zip(f.inputs, gxs):
                if input.grad is None:
                    input.grad = gx
                else:
                    input.grad = input.grad + gx

                if input.creator is not None:
                    add_func(input.creator)

            if not retain_grad:
                # retain_gradがTrueの時、勾配を保持
                # Falseの時、勾配を破棄する
                for output in f.outputs:
                    output().grad = None  # y is weakref

In [49]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()
print(y.grad, t.grad)  # None None
print(x0.grad, x1.grad)  # 2.0 1.0

None None
2.0 1.0


### 18.2 Functionクラスの復習
```Py
class Function:
    def __call__(self, *inputs):
        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([input.generation for input in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs # 入力を保持
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]
    ...
```
`self.inputs`は、逆伝播の計算時に使用される。つまり、推論時には逆伝播は行わないため、保持する必要はない。<br>
また、計算の<strong>つながり</strong>も作る必要はない。

### 18.3 Configクラスによる切り替え
「逆伝播の有効モード」と「逆伝播の無効モード」を切り替えるConfigクラスを作る。

In [50]:
class Config:
    enable_backprop = True

In [51]:
class Function:
    def __call__(self, *inputs):
        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([input.generation for input in inputs]) # 世代の設定
            for output in outputs:
                output.set_creator(self) # つながりの設定
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 18.4 モードをきりかえる

In [52]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)

In [53]:
%load_ext memory_profiler

In [54]:
Config.enable_backprop = True
def train():
    x = Variable(np.ones((100, 100, 100)))
    y = square(square(square(x)))
    #y.backward()

%memit train()

peak memory: 123.51 MiB, increment: 27.74 MiB


In [55]:
def valid():
    Config.enable_backprop = False
    x = Variable(np.ones((100, 100, 100)))
    y = square(square(square(x)))

%memit valid()

peak memory: 118.54 MiB, increment: 22.56 MiB


### 18.5 with文による切り替え

In [56]:
import contextlib

`@contextlib.contextmanager`というデコレータを付けることで、コンテキスト(文脈)を判断する関数をつくることができる。<br>
関数内では、`yield`の前に前処理を書き、`yield`の後に後処理を書く。

In [57]:
@contextlib.contextmanager
def config_test():
    print('start') # 前処理
    try:
        yield
    finally:
        print('done') # 後処理

with config_test():
    print('process...')

start
process...
done


In [58]:
@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name) # Configクラスのnameで指定した変数の値を得る
    setattr(Config, name, value) # Configクラスのnameで指定した変数を更新する
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [59]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

毎回`using_config('enable_backprop', False)`と書くのは面倒なので、`no_grad()`関数を作る

In [60]:
def no_grad():
    return using_config('enable_backprop', False)

In [61]:
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

## ステップ１９　変数を使いやすく

In [62]:
class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)

        add_func(self.creator)

        while funcs:
            f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for input, gx in zip(f.inputs, gxs):
                if input.grad is None:
                    input.grad = gx
                else:
                    input.grad = input.grad + gx

                if input.creator is not None:
                    add_func(input.creator)

            if not retain_grad:
                for output in f.outputs:
                    output().grad = None  # y is weakref

### 19.1 変数に名前を付ける
Variableクラスに`name`というインスタンス変数を追加する。
```Py
class Variable:
    def __init__(self, data, name=None): # name変数の追加
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name # name変数の設定
        self.grad = None
        self.creator = None
        self.generation = 0
```

### 19.2 ndarrayのインスタンス変数
shapeというメソッドを実装する。メソッドの前に`@property`という一行を入れることで、shapeメソッドはインスタンス変数としてアクセスできる。
```Py
class Variable:
    ...
    @property
    def shape(self):
        return self.data.shape
```
上記と同様に、３つのインスタンス変数を新たに追加する。

```Py
class Variable:
    ...
    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

```





In [63]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]), name='x_input')
print(x.shape) # x.shapeではなく、x.shapeでアクセスできる

(2, 3)


In [64]:
print(x.ndim)
print(x.dtype)
print(x.size)
print(x.name)

2
int64
6
x_input


### 19.3 len関数とprint関数
```Py
class Variable:
    ...
    def __len__(self):
        return len(self.data)
```
上記のように、__len__という特殊メソッドを実装すれば、Variableインスタンスに対しても、len関数を使えるようになる。

In [65]:
print(len(x))

2


In [66]:
x = Variable(np.array([1, 2, 3]))
print(x)

x = Variable(None)
print(x)

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x)

variable([1 2 3])
variable(None)
variable([[1 2 3]
          [4 5 6]])


## ステップ２０　演算子のオーバーロード(1)

### 20.1 Mulクラスの実装
　掛け算の実装を行う。$y=x_0 \times x_1$の時、$\frac{\partial y}{\partial x_0} = x_1$、$\frac{\partial y}{\partial x_1} = x_0$である。

In [67]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

In [68]:
def mul(x0, x1):
    return Mul()(x0, x1)

In [69]:
Config.enable_backprop = True
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c)
y.backward()

print(y)
print(a.grad)
print(b.grad)
print(c.grad)

variable(7.0)
2.0
3.0
1.0


上記のコードは、$y=a \times b + c $を実行している。よって、$\frac{\partial y}{\partial a} = b, \ \frac{\partial y}{\partial b} = a, \ \frac{\partial y}{\partial c} = 1$となる。<br>
しかし、`y = add(mul(a, b), c)`と書くのは面倒で、`y = a * b + c`のように書けた方が好ましい。

### 20.2 演算子のオーバーロード
掛け算の特殊メソッドは、`__mul__(self, other)`である。例えば、　`y = a * b`とすると、`a`が`self`、`b`が`other`となる。<br>
同様に、足し算の特殊メソッドは、`__add__(self, other)`である。

In [70]:
Variable.__add__ = add
Variable.__mul__ = mul

In [71]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a, b), c)
y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


## ステップ２１　演算子のオーバーロード(2)
### 21.1 ndarrayと一緒に使う

In [72]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [73]:
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(input) for input in inputs]

        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([input.generation for input in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [74]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy
    
def add(x0, x1):
    return Add()(x0, x1)

Variable.__add__ = add

In [75]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
y

variable(5.0)

### 21.2 floatやintと一緒に使う
Pythonのintやfloat、またnp.float64やnp.int64といった型と組み合わせて使えるように変更する<br>
以前作ったas_array関数を使う

In [76]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [77]:
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

In [78]:
Variable.__add__ = add

In [79]:
x = Variable(np.array(2.0))
y = x + 3.0
y

variable(5.0)

### 21.3 問題点１：左項がfloatやintの場合

In [80]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

In [81]:
Variable.__mul__ = mul

In [82]:
y = 2.0 * x

TypeError: unsupported operand type(s) for *: 'float' and 'Variable'

`*`のような２つの項を使う演算子の場合、左項の場合は、`__mull__`メソッド、右項の場合は、`__rmul__`メソッドが呼ばれる。足し算なども同様である。

In [83]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [84]:
y = 3.0 * x + 1.0
print(y)

variable(7.0)


### 21.4 問題点２：左項がndarrayインスタンスの場合

現状では、ndarrayが左項、Variableが右項の場合、ndarrayが優先され、`__add__`メソッドが呼ばれる。<br>
ここでは、右項にあるVariableインスタンスの`__radd__`メソッドを呼ばれるようにしたい。<br>
そこで、Variableインスタンスの<strong>演算子の優先度</strong>を上げて対応する。

In [85]:
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x
y # ndarrayの__add__メソッドが呼ばれ、出力がndarrayになっている

array([variable([3.])], dtype=object)

In [86]:
class Variable:
    __array_priority__ = 200 # 演算子の優先度を上げる

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)

        add_func(self.creator)

        while funcs:
            f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for input, gx in zip(f.inputs, gxs):
                if input.grad is None:
                    input.grad = gx
                else:
                    input.grad = input.grad + gx

                if input.creator is not None:
                    add_func(input.creator)

            if not retain_grad:
                for output in f.outputs:
                    output().grad = None  # y is weakref

In [87]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [88]:
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x
y

variable([3.])

## ステップ２２　演算子のオーバーロード(3)
以下の６つについて、新たに演算子を追加する。<br>
<br>
|特殊メソッド|例|
|:--|:--|
|`__neg__(self)`|`-self`|
|`__sub__(self, other)`|`self - other`|
|`__rsub__(self, other)`|`other - self`|
|`__truediv__(self, other)`|`self / other`|
|`__rtruediv__(self, other)`|`other - self`|
|`__pow__(self, other)`|`self ** other`|

### 22.1 負数
$y=-x$の時、$\frac{\partial y}{\partial x} = -1 $と求められるため。上流の微分に対して、-1倍して下流に流せば良い

In [89]:
def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy # -1倍


def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

### 22.2 引き算
$y=x_0-x_1$の時、$\frac{\partial y}{\partial x_0} = 1,\ \frac{\partial y}{\partial x_1} = -1$となる。そのため、逆伝播は、上流から伝わる微分に対して、そのまま流すのが$x_0$の微分となり、-1倍にしたものが$x_1$の微分となる。

In [90]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy


def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

Variable.__sub__ = sub

In [91]:
x = Variable(np.array(2.0))
y = -x
print(y)

variable(-2.0)


`y = 2.0 + x`のような場合には、うまくいかないので、`__rsub__`を実装する

In [92]:
def rsub(x0, x1):
    x1 = as_array(x1)
    return sub(x1, x0) # x1, x0を入れ替え

Variable.__rsub__ = rsub

In [93]:
y1 = 2.0 - x
y2 = x - 1.0
print(y1)
print(y2)  

variable(0.0)
variable(1.0)


### 22.3 割り算
$y=x_0/x_1$の時、$\frac{\partial y}{\partial x_0} = 1/x_1, \ \frac{\partial y}{\partial x_1} = -x_0/(x_1)^2$となる。

In [94]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1


def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)


def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0)

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [95]:
y1 = 3.0 / x
print(y1) 

y2 = x / 1
print(y2)

variable(1.5)
variable(2.0)


### 22.4 累乗
$y=x^c$の時、$\frac{\partial y}{\partial x} = cx^{c-1}$となる。指数のcについての微分は考慮しないとする。

In [96]:
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx
    
def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow

In [97]:
class Config:
    enable_backprop = True
    
y = x ** 3
y.backward()
print(y)
print(x.grad)

variable(8.0)
12.0


### ステップ２３　パッケージとしてまとめる

In [98]:
import numpy as np
from leopard import Variable

In [99]:
x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()

print(y)
print(x.grad)

variable(16.0)
variable(8.0)


## ステップ２４ 複雑な関数の微分

### 24.1 Sphere関数
Sphere関数は、$z=x^2+y^2$という数式になる。この時、$\frac{\partial z}{\partial x} = 2x, \ \frac{\partial z}{\partial y} = 2y$となる。$(x,y)=(1.0,1.0)$における微分を求めてみる。

In [100]:
def sphere(x, y):
    return x ** 2 + y ** 2

In [101]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = sphere(x, y)
z.backward()
print(x.grad, y.grad)

variable(2.0) variable(2.0)


### 24.2 matyas関数
matyas関数は、$z=0.26(x^2+y^2) - 0.48xy$と表される。これを実装してみる。

In [102]:
def matyas(x, y):
    return 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y

In [103]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = matyas(x, y)
z.backward()
print(x.grad, y.grad)

variable(0.040000000000000036) variable(0.040000000000000036)


### 24.3 Goldstein-Price関数
Goldstein-Price関数は、数式で次のように表される。これを実装してみる。<br>
$[1+(x+y+1)^2(19-14x+3x^2-14y+6xy+3y^2)]\times$<br>
$[30+(2x-3y)^2(18-32x+12x^2+48y-36xy+27y^2)]$

In [104]:
def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z

In [105]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)

variable(-5376.0) variable(8064.0)


# 第２ステージ終了